In [1]:
# load the first simple geometry
import fiona
from shapely.geometry import *
# from shapely import speedups
# speedups.enable()
import json

In [2]:
# coords = ((0., 0.), (0., 1.), (1., 1.), (1., 0.), (0., 0.))
# coords = ((0., 0.), (0., 1.), (1., 1.), (1., 0.),)
# # LinearRing?
# # l = LinearRing(coordinates=coords)
# # Polygon?
# # print(l)
# polygon = Polygon(shell=coords)
# # polygon.area

In [3]:
# c = fiona.open('VT/SA2012VT_External.shp','r')
c = fiona.open('CITIES/cb_2013_us_ua10_500k.shp','r')
coordinates = []
print(c) # Collection object
print(len(c)) # 8587
print(len(list(c)))
for city in list(c): 
    print(city)
    # print(city['properties']['AREA_NAME'])
    if city['geometry']['type'] == 'Polygon':
        # print(city)
        coordinates = city['geometry']['coordinates'][0]
        break
p = Polygon(coordinates)
print(p)
pt = Point([-97.58,27.94])
p.contains(pt)

<open Collection 'CITIES/cb_2013_us_ua10_500k.shp:cb_2013_us_ua10_500k', mode 'r' at 0x10fdc8290>
3601
3601
POLYGON ((-110.642771 32.596403, -110.640142 32.599216, -110.641214 32.602232, -110.634806 32.609506, -110.632637 32.610398, -110.631214 32.612955, -110.631592 32.61415, -110.629253 32.615778, -110.631672 32.618789, -110.630124 32.619622, -110.628183 32.618931, -110.6286 32.617863, -110.626075 32.614449, -110.628104 32.614184, -110.617735 32.600968, -110.63069 32.599631, -110.632248 32.597527, -110.635779 32.59682, -110.637135 32.599611, -110.639453 32.597127, -110.638702 32.595627, -110.641772 32.594369, -110.642771 32.596403))


False

In [26]:
c = fiona.open('NPS/nps_boundary.shp','r')
coordinates = []
print(c) # Collection object
print(len(c)) # 8587
print(len(list(c)))
polygons = 0
multipolygons = 0
for city in list(c): 
    # print(city)
    print("{},{}".format(city['properties']['PARKNAME'],city['properties']['UNIT_NAME']))
    if city['geometry']['type'] == 'Polygon':
        polygons += 1
        # print(city)
        coordinates = city['geometry']['coordinates'][0]
    if city['geometry']['type'] == 'MultiPolygon':
        multipolygons += 1
p = Polygon(coordinates)
# print(p)
pt = Point([-97.58,27.94])
p.contains(pt)
print("polygon count: {0}".format(polygons))
print("multipolygon count: {0}".format(multipolygons))

<open Collection 'NPS/nps_boundary.shp:nps_boundary', mode 'r' at 0x115c31050>
410
410
American Memorial Park,American Memorial Park
Crater Lake,Crater Lake National Park
Ebey's Landing,Ebey's Landing National Historical Reserve
Lake Chelan,Lake Chelan National Recreation Area
Lake Roosevelt,Lake Roosevelt National Recreation Area
Lewis and Clark,Lewis and Clark National Historical Park
North Cascades,North Cascades National Park
National Park of American Samoa,National Park of American Samoa National Park
Olympic,Olympic National Park
Ross Lake,Ross Lake National Recreation Area
Santa Monica Mountains,Santa Monica Mountains National Recreation Area
Tule Springs Fossil Beds,Tule Springs Fossil Beds National Monument
None,Acadia National Park
None,Ala Kahakai National Historic Trail
None,Amistad National Recreation Area
None,Appalachian National Scenic Trail
None,Assateague Island National Seashore
None,Big Bend National Park
None,Big Cypress National Preserve
None,Big Hole National Bat

In [6]:
# print(len(coordinates))
# for c in coordinates:
#     print(len(c))
#     print(len(c[0]))
# # there are 12, length one lists
# # since the lists have length 1, I think this means
# # that there are no holes
# # and that these are 12 disjoint areas

In [7]:
# The MultiPolygon constructor takes a sequence of exterior ring
# and hole list tuples: [((a1, ..., aM), [(b1, ..., bN), ...]), ...].
exterior_ring = ((1,1),(0,1),(0,0),(1,0))
interior_hole1 = ((.3,.3),(.1,3),(.1,.1),(.3,.1))
interior_hole2 = ((.9,.9),(.7,9),(.7,.7),(.9,.7))
p = MultiPolygon([(exterior_ring,[interior_hole1,interior_hole2]),(exterior_ring,[])])

In [8]:
try:
    p = MultiPolygon(coordinates)
except:
    print('nope')

nope


In [10]:
# coordinates_w_empty_holes = [(tuple(c[0]),()) for c in coordinates]

In [12]:
# p = MultiPolygon(coordinates_w_empty_holes)

In [13]:
pt = Point([-97.58,27.94])
p.contains(pt)

False

In [27]:
# load all of the geometries
c = fiona.open('NPS/nps_boundary.shp','r')
polygonList = []
nameList = []
polygonCount = 0
multiPolygonCount = 0
for city in list(c):
    nameList.append(city['properties']['UNIT_NAME'])
    if city['geometry']['type'] == 'Polygon':
        polygonCount += 1
        # print(city)
        coordinates = city['geometry']['coordinates']
        polygonList.append(Polygon(coordinates[0]))
    elif city['geometry']['type'] == 'MultiPolygon':
        multiPolygonCount += 1
        coordinates = city['geometry']['coordinates']
        # check that there aren't any holes:
        # for c in coordinates:
            # if len(c) != 1:
                # print('{0} has holes...in different area codes'.format(city['properties']['NAME10']))
                # print(c)
        coordinates_w_holes = [(tuple(c[0]),()) if len(c) == 1 else (tuple(c[0]),(c[1:])) for c in coordinates]
        polygonList.append(MultiPolygon(coordinates_w_holes))
    else:
        print('unknown geometry ' % city['geometry']['type'])
# p = MultiPolygon(coordinates)
# print(p)
print('done')

done


In [28]:
print(polygonCount)
print(multiPolygonCount)

206
204


In [35]:
nameList[12]

u'Acadia National Park'

In [31]:
nameList

[u'American Memorial Park',
 u'Crater Lake National Park',
 u"Ebey's Landing National Historical Reserve",
 u'Lake Chelan National Recreation Area',
 u'Lake Roosevelt National Recreation Area',
 u'Lewis and Clark National Historical Park',
 u'North Cascades National Park',
 u'National Park of American Samoa National Park',
 u'Olympic National Park',
 u'Ross Lake National Recreation Area',
 u'Santa Monica Mountains National Recreation Area',
 u'Tule Springs Fossil Beds National Monument',
 u'Acadia National Park',
 u'Ala Kahakai National Historic Trail',
 u'Amistad National Recreation Area',
 u'Appalachian National Scenic Trail',
 u'Assateague Island National Seashore',
 u'Big Bend National Park',
 u'Big Cypress National Preserve',
 u'Big Hole National Battlefield',
 u'Big South Fork National River and Recreation Area',
 u'Blue Ridge Parkway',
 u'Bluestone National Scenic River',
 u'Canyon de Chelly National Monument',
 u'Cape Cod National Seashore',
 u'Cape Hatteras National Seashore',

In [16]:
pt = Point([-97.58,27.94])
for i,park in enumerate(polygonList):
    if park.contains(pt):
        print(i)
        print(nameList[i])
        break

In [17]:
def parkID(polygonList,pt):
    for i,park in enumerate(polygonList):
        if park.contains(pt):
            return i
    return -1

In [18]:
%time
locID = parkID(polygonList,pt)
print(locID)

CPU times: user 2 µs, sys: 0 ns, total: 2 µs
Wall time: 3.81 µs
-1


In [22]:
%time
f = open('tweets/2015-01-01-12-00.json','r')
count = 0
for line in f:
    count += 1
    if count > 40000:
        break
    tweet = json.loads(line)
    # print(tweet['coordinates']['coordinates'])
    if 'coordinates' in tweet:
        if tweet['coordinates'] is not None:
            pt = Point(tweet['coordinates']['coordinates'])
            loc = parkID(polygonList,pt)
            if loc > -1:
                print(nameList[loc])
        #     else:
        #         print('not found')

CPU times: user 2 µs, sys: 1 µs, total: 3 µs
Wall time: 6.91 µs
